# Exploratory Data Analysis for Fitbit Health

## Export json files into multiple small csv files to store dataset

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
import pandas as pd
import gzip
data = []
#with gzip.open('endomondoHR.json.gz') as f:
with open('/content/drive/MyDrive/endomondoHR_proper.json') as f:
    i = 1
    for l in f:
        if i % 50000!=0:
            data.append(eval(l))
            i+=1
        else:
            path = '/content/drive/MyDrive/'+str(i//50000)+'pro.csv'
            pd.DataFrame(data).to_csv(path)
            i+=1
            data = []
            print(i)

50001
100001
150001


In [ ]:
import gzip
from google.colab import drive
import pandas as pd

#drive.mount('/content/gdrive/')

data = []
with gzip.open('/content/drive/MyDrive/endomondoMeta.json.gz') as f:
    i = 1
    for l in f:
        if i % 100000!=0:
            data.append(eval(l))
            i+=1
        else:
            path = '/content/drive/MyDrive/'+str(i//100000)+'.csv'

            pd.DataFrame(data)[['weather','id','hydration']].to_csv(path)
            i+=1
            data = []
            print(i)

100001
200001
300001
400001
500001
600001
700001
800001
900001


## Import Spark 

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.91.39 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u312-b07-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.91.39 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Exception: ignored

## EndomondoHR
EndomondoHR
- The dataset contains 149997 rows and 12 columns
- Missingness
  - Only `speed` column contains missing data of 121634 missing entries
- `sport`
  - There are total of 41 categories in `sport`
  - The top 10 most popular sports are 
    1. run (63608)
    2. bike (63093)
    3. mountain bike (10033)
    4. bike - transport (6993)
    5. indoor cycling (1565)
    6. walk (1166)
    7. orienteering (814)
    8. cross-country skiing (746)
    9. core stability training (406)
    10. fitness walking (278)
- `gender`
  - `gender` distribution
    - unknown	906
    - female	9537
    - male	139554
  - `gender` might influence the selection of `sport`
    - While the top 10 most popular `sport` for male and female are quite similar, the ranking of the `sport` is different
    - Male engage in 41 unique `sport` activities, while female only engage in 22 unique `sport` activities 
- `userId`

EndomondoMeta 
- The dataset contains 899991 rows 4 columns



## EndomondoMeta Dataset


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
# Endomondo HR Files
pro1 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/1pro.csv")
pro2 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/2pro.csv")
pro3 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/3pro.csv")
dfhr = pro1.union(pro2).union(pro3)
print((dfhr.count(), len(dfhr.columns)))

In [ ]:
# 1. Missing values
dfhr.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dfhr.columns]).show()

In [ ]:
# 2. Sport Categories
from pyspark.sql.functions import countDistinct
dfcount = dfhr.groupBy("sport").count()
dfcount = dfcount.sort(col("count"))
print(dfcount.count())
dfcount.tail(10)

In [ ]:
# 3. Gender
dfgender = dfhr.groupBy("gender").count()
dfgender

In [ ]:
# Plot gender distribution
pdgender = dfgender.toPandas()
pdgender = pdgender.set_index('gender')
pdgender.plot.pie(y='count')

In [ ]:
# Gendere vs Sport
#dfinter = dfhr.groupBy("sport", "gender").count().toPandas()
dfinter.loc[dfinter['gender'] == 'male'].sort_values('count',ascending = False).head(10)


In [ ]:
dfinter.loc[dfinter['gender'] == 'female'].sort_values('count',ascending = False).head(10)

In [ ]:
dfmale = dfinter.loc[dfinter['gender'] == 'male']
dffemale = dfinter.loc[dfinter['gender'] == 'female']
print(dfmale.shape)
print(dffemale.shape)

In [ ]:
# 4. Unique userId
dfuser = dfhr.groupBy("userId").count()
dfuser = dfuser.sort(col("count"))
print(dfuser.count())
dfuser.tail(10)

In [ ]:
dfhr

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34447)
Traceback (most recent call last):
  File "/content/spark-3.1.1-bin-hadoop3.2/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-3.1.1-bin-hadoop3.2/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: ignored

DataFrame[_c0: string, longitude: string, altitude: string, latitude: string, sport: string, id: string, heart_rate: string, gender: string, timestamp: string, url: string, userId: string, speed: string]

In [ ]:
# Endomondo Meta Files
meta1 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/1.csv")
meta2 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/2.csv")
meta3 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/3.csv")
meta4 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/4.csv")
meta5 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/5.csv")
meta6 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/6.csv")
meta7 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/7.csv")
meta8 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/8.csv")
meta9 = spark.read.option("header","true").csv("/content/gdrive/MyDrive/9.csv")
dfmeta = meta1.union(meta2).union(meta3).union(meta4).union(meta5).union(meta6).union(meta7).union(meta8).union(meta9)
print((dfmeta.count(), len(dfmeta.columns)))

(899991, 4)


['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11']

# Previous Code

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from pyspark.sql import *
path = '/content/endomondoMeta.json'
df = spark.read.json(path)

In [ ]:
#df = spark.read.option("multiline","true").json('/content/drive/MyDrive/endomondoMeta.json.gz')
a=spark.read.json('/content/drive/MyDrive/endomondoMeta.json.gz', multiLine=True)

In [ ]:
import gzip
import shutil
with gzip.open('/content/drive/MyDrive/endomondoMeta.json.gz', 'rb') as f_in:
    with open('endomondoMeta.json', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
from pyspark.sql.functions import col,sum
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()

In [ ]:
with gzip.GzipFile('/content/drive/MyDrive/endomondoMeta.json.gz', 'r') as fin:   
  for line in fin:
    print(json.loads(line))

In [ ]:
from pyspark.sql.functions import mean
dataframe.select(mean("Duration")).show()

Goal: recommend sport/length with different weather?
We want to take a look at the unique value of each column<p>
Some potential questions: at which weather did users exercise the most?
Gender affects sport? <n>
We have 174 users, so we can use individual as prediction